# A* Algorithm Implmentation: Closest Store by BART

**imports**

In [173]:
import neo4j
import math
import numpy as np
import pandas as pd
import re
import random

import psycopg2
from scipy import spatial
from geographiclib.geodesic import Geodesic

**Driver Connection**

In [174]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)
cursor = connection.cursor()
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

**Helper Functions**

In [175]:
## Pandas Functions

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [176]:
## Generic Neo4j Functions

def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query) 
    
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

In [177]:
## Create Nodes

def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    
def my_neo4j_create_customer_node(customer_id):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Customer {
                        customer_id: $customer_id
                    })
    
    """
    
    session.run(query, customer_id=customer_id)
    
def my_neo4j_create_locker_node(locker_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Locker {
                        locker_name: $locker_name, 
                    })
    
    """
    
    session.run(query, locker_name=locker_name)

In [178]:
## Create One-Way Relationships 

def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

def create_relationship_one_way_customer_station(from_customer, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Customer), 
          (to:Station)
    WHERE from.customer_id = $from_customer and to.name = $to_station
    CREATE (from)-[:DIST {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_customer=from_customer, to_station=to_station, weight=weight)
    
def create_relationship_one_way_locker_station(from_station, locker_name, weight):
    """
    create a relationship one way between two stations with a weight
    note: may need to add locker station
    """
    
    query = """
    
    MATCH (from:Station), 
          (to:Locker)
    WHERE from.station_name = $from_station and to.locker_name = $locker_name 
    CREATE (from)-[:DIST {weight: $weight}]->(to)
    
    """
    session.run(query, from_station=from_station, locker_name=locker_name, weight=weight)


In [179]:
# Create Two-Way Relationships 
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)



    
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)




In [180]:
## Distance/Random Latitude Functions 

def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

def random_point(p, m): 
    l, r, t, b = my_calculate_box(p, m)
    
    random_latitude = random.uniform(l[0], r[0])
    random_longitude = random.uniform(b[1], t[1])
        
    return (random_latitude, random_longitude)


In [181]:
## Graph Algorithms
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1

## Graph Creation plan 
- Customer nodes: 
    - randomly generate latitudes and longitudes within the customer's zip code 
    - find the closest bart station to them, create a connection 
    Note: only using bay area zip codes
- Store nodes: 
    - find the closest bart station to the store 

### Customer Zip Code -> Random (lat, long) pairs
- note: need to filter for bay area zip codes only 

In [136]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT customers.customer_id, customers.zip, zip_codes.latitude, zip_codes.longitude
FROM customers
left join zip_codes 
on customers.zip = zip_codes.zip 
where customers.zip like '9%'

"""

customers = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
customers['zip'].value_counts()

98117    315
98125    296
94602    260
94530    258
98119    257
        ... 
94565      1
94588      1
94070      1
94516      1
94957      1
Name: zip, Length: 258, dtype: int64

In [137]:
random_locs = customers.apply(lambda x: random_point((x['latitude'], x['longitude']), 1), axis=1)
customers["random_lat"] = random_locs.apply(lambda x: x[0])
customers["random_long"] = random_locs.apply(lambda x: x[1])
rand_customers_loc = customers[['customer_id', 'zip', 'random_lat', 'random_long']]
rand_customers_loc

,customer_id,zip,random_lat,random_long
0,1,94609,37.834299,-122.2643
1,2,94609,37.834299,-122.2643
2,3,94609,37.834299,-122.2643
3,4,94609,37.834299,-122.2643
4,5,94609,37.834299,-122.2643
...,...,...,...,...
15347,15348,98421,47.259198,-122.3995
15348,15349,98421,47.259198,-122.3995
15349,15350,98421,47.259198,-122.3995
15350,15351,98416,47.262498,-122.4812


## Get Stations and Coordinates

In [138]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT *
FROM stations

"""

stations = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
stations.head()

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0


## Building customers nodes, and their relations to their closest stations

In [139]:
tree = spatial.KDTree(stations[['latitude', 'longitude']])
dists, inds = tree.query(rand_customers_loc[['random_lat', 'random_long']])

In [140]:
rand_customers_loc['distance_miles'] = dists
rand_customers_loc['travel_time_secs'] = (dists / 2.5) * 3600

rand_customers_loc['closest_station'] = stations.loc[inds,'station'].values
rand_customers_loc

/tmp/ipykernel_24/3608011209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rand_customers_loc['distance_miles'] = dists


,customer_id,zip,random_lat,random_long,distance_miles,travel_time_secs,closest_station
0,1,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur
1,2,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur
2,3,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur
3,4,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur
4,5,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur
...,...,...,...,...,...,...,...
15347,15348,98421,47.259198,-122.3995,9.251539,13322.215963,Pittsburg
15348,15349,98421,47.259198,-122.3995,9.251539,13322.215963,Pittsburg
15349,15350,98421,47.259198,-122.3995,9.251539,13322.215963,Pittsburg
15350,15351,98416,47.262498,-122.4812,9.259214,13333.267701,Pittsburg


In [141]:
print("Number of Customers Closest to Station")
rand_customers_loc.loc[:, 'closest_station'].value_counts()

Number of Customers Closest to Station


Pittsburg               7214
El Cerrito del Norte     844
Downtown Berkeley        578
Civic Center             469
Richmond                 458
Fruitvale                440
Orinda                   400
MacArthur                385
Lake Merritt             375
19th Street              337
Lafayette                318
Walnut Creek             306
El Cerrito Plaza         294
Rockridge                290
North Berkeley           246
San Leandro              239
West Oakland             203
Embarcadero              189
Bay Fair                 173
Montgomery Street        168
16th Street Mission      162
Balboa Park              153
24th Street Mission      145
OAK                      128
Glen Park                127
Pleasant Hill             97
Daly City                 77
Powell Street             74
Coliseum                  67
Millbrae                  55
North Concord             48
Colma                     45
South San Francisco       42
Concord                   36
Castro Valley 

## Rebuilding the BART Graph 

In [142]:
## Starting Graph
## Wiping out the database 
my_neo4j_wipe_out_database()
## Nodes Post Wipeout
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


In [143]:
## Adding Station Departure and Arrival Nodes 
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)

## Verify the number of nodes 
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


In [144]:
query = """

SELECT station, line
FROM lines
ORDER BY station, line

"""

cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    station = row[0]
    line = row[1]
    node = line + ' ' + station
    my_neo4j_create_node(node)
    my_neo4j_create_relationship_one_way('depart ' + station, node, 0)
    my_neo4j_create_relationship_one_way(node, 'arrive ' + station, 0)
    
## Verify the number of nodes 
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


In [145]:
query = """

WITH line_to_line AS (
    SELECT l1.station, l1.line AS from_line, l2.line AS to_line
    FROM lines l1 JOIN lines l2 ON l1.station = l2.station 
    WHERE l1.line != l2.line)
    
SELECT stations.station, from_line, to_line, transfer_time
FROM line_to_line JOIN stations ON line_to_line.station = stations.station

"""

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    my_neo4j_create_relationship_one_way(from_line + ' ' + station, to_line + ' ' + station, transfer_time) 
    
my_neo4j_number_nodes_relationships()


-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


In [146]:
query = """

WITH line_to_line AS(
    SELECT l1.line, l1.station AS "from station", l2.station AS "to station"
    FROM lines l1 JOIN lines l2 ON l1.line = l2.line
    WHERE l1.station != l2.station AND (l1.sequence + 1) = (l2.sequence)
)

SELECT line, "from station", "to station", travel_time AS "travel time in seconds"
FROM line_to_line, travel_times
WHERE ("from station" = station_1 AND "to station" = station_2) OR ("from station" = station_2 AND "to station" = station_1 )
ORDER BY line, "from station", "to station"

"""

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    line = row[0]
    from_station = row[1]
    to_station = row[2]
    travel_time = int(row[3])
    my_neo4j_create_relationship_two_way(line + ' ' + from_station, line + ' ' + to_station, travel_time)
    
my_neo4j_number_nodes_relationships()


-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


## Testing Shortest Path on Rebuilt Graph 

In [147]:
my_neo4j_shortest_path('depart Dublin', 'arrive Antioch')


--------------------------------
   Total Cost:  5813
   Minutes:  96.9
--------------------------------
depart Dublin, 0, 0
blue Dublin, 0, 0
blue West Dublin, 180, 180
blue Castro Valley, 600, 780
blue Bay Fair, 240, 1020
blue San Leandro, 240, 1260
blue Coliseum, 240, 1500
orange Coliseum, 54, 1554
orange Fruitvale, 240, 1794
orange Lake Merritt, 300, 2094
orange 12th Street, 180, 2274
orange 19th Street, 120, 2394
orange MacArthur, 180, 2574
yellow MacArthur, 59, 2633
yellow Rockridge, 240, 2873
yellow Orinda, 300, 3173
yellow Lafayette, 300, 3473
yellow Walnut Creek, 300, 3773
yellow Pleasant Hill, 120, 3893
yellow Concord, 360, 4253
yellow North Concord, 180, 4433
yellow Pittsburg, 360, 4793
yellow Pittsburg Center, 600, 5393
yellow Antioch, 420, 5813
arrive Antioch, 0, 5813


### Creating Customer -> Departure Station Relationships 

In [148]:
rand_customers_loc.head()

,customer_id,zip,random_lat,random_long,distance_miles,travel_time_secs,closest_station
0,1,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur
1,2,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur
2,3,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur
3,4,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur
4,5,94609,37.834299,-122.2643,0.006732,9.693573,MacArthur


**Creating Customer Nodes**

In [149]:
## Creating Customer Nodes 
connection.rollback()
my_neo4j_number_nodes_relationships()
cust_ids = rand_customers_loc['customer_id'].values
print(214 + len(cust_ids))
for i in range(len(cust_ids)):
    my_neo4j_create_customer_node(cust_ids[i])
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------
15566
-------------------------
  Nodes: 15566
  Relationships: 652
-------------------------


**Creating Customer -> Departure Station relationships** 
- note: travel time in seconds was used for the weights

In [150]:
connection.rollback()

for i in range(len(dists)):
    
    customer_node = cust_ids[i]
    departure_station = 'depart ' + rand_customers_loc['closest_station'].values[i]
    travel_time_weight = rand_customers_loc['travel_time_secs'].values[i]
    
    create_relationship_one_way_customer_station(customer_node, departure_station, travel_time_weight)

In [151]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 15566
  Relationships: 16004
-------------------------


### Creating Departure Station -> Pickup Locker Relationships 

In [152]:
stations.head()

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0


In [153]:
locker_stations = ['12th Street',
                 '16th Street Mission',
                 '19th Street',
                 '24th Street Mission',
                 'Bay Fair',
                 'Civic Center',
                 'Coliseum',
                 'Embarcadero',
                 'Fruitvale',
                 'Lake Merritt',
                 'MacArthur',
                 'Montgomery Street',
                 'Powell Street',
                 'San Leandro',
                 'West Oakland']

locker_df = stations[stations["station"].isin(locker_stations)]
locker_df

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
7,Bay Fair,37.697000,-122.126500,63
10,Civic Center,37.779861,-122.413498,325
11,Coliseum,37.753611,-122.196944,54
19,Embarcadero,37.793056,-122.397222,304
21,Fruitvale,37.774800,-122.224100,279
25,Lake Merritt,37.797773,-122.266588,309


In [154]:
## Creating Locker Nodes 
connection.rollback()
my_neo4j_number_nodes_relationships()

for i in range(locker_df.shape[0]):
    locker_name = locker_df['station'].values[i] + " AGM Pickup Locker"
    locker_location = locker_df['station'].values[i]
    my_neo4j_create_locker_node(locker_name, locker_location)
my_neo4j_number_nodes_relationships()


-------------------------
  Nodes: 15566
  Relationships: 16004
-------------------------
-------------------------
  Nodes: 15581
  Relationships: 16004
-------------------------


**Creating Arrival Station -> Pickup Locker relationships** 
- note: travel time = 120 seconds assuming non-zero time to account for pickup time

In [158]:
connection.rollback()
my_neo4j_number_nodes_relationships()

for i in range(locker_df.shape[0]):
    create_relationship_one_way_locker_station("arrive " + locker_df['station'].values[i], locker_df['station'].values[i] + " AGM Pickup Locker", 120)
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 15581
  Relationships: 16004
-------------------------
-------------------------
  Nodes: 15581
  Relationships: 16019
-------------------------


In [162]:
my_neo4j_shortest_path('depart Dublin', 'arrive Antioch')


--------------------------------
   Total Cost:  5813
   Minutes:  96.9
--------------------------------
depart Dublin, 0, 0
blue Dublin, 0, 0
blue West Dublin, 180, 180
blue Castro Valley, 600, 780
blue Bay Fair, 240, 1020
blue San Leandro, 240, 1260
blue Coliseum, 240, 1500
orange Coliseum, 54, 1554
orange Fruitvale, 240, 1794
orange Lake Merritt, 300, 2094
orange 12th Street, 180, 2274
orange 19th Street, 120, 2394
orange MacArthur, 180, 2574
yellow MacArthur, 59, 2633
yellow Rockridge, 240, 2873
yellow Orinda, 300, 3173
yellow Lafayette, 300, 3473
yellow Walnut Creek, 300, 3773
yellow Pleasant Hill, 120, 3893
yellow Concord, 360, 4253
yellow North Concord, 180, 4433
yellow Pittsburg, 360, 4793
yellow Pittsburg Center, 600, 5393
yellow Antioch, 420, 5813
arrive Antioch, 0, 5813


In [169]:
customer_id = 2
pickup_locker = "Embarcadero AGM Pickup Locker"

"""
['12th Street',
 '16th Street Mission',
 '19th Street',
 '24th Street Mission',
 'Bay Fair',
 'Civic Center',
 'Coliseum',
 'Embarcadero',
 'Fruitvale',
 'Lake Merritt',
 'MacArthur',
 'Montgomery Street',
 'Powell Street',
 'San Leandro',
 'West Oakland']
"""

my_neo4j_shortest_path(customer_id, "Embarcadero")

my_neo4j_shortest_path('depart Dublin', "arrive Embarcadero")

my_neo4j_shortest_path(customer_id, pickup_locker)


--------------------------------
   Total Cost:  2820
   Minutes:  47.0
--------------------------------
depart Dublin, 0, 0
blue Dublin, 0, 0
blue West Dublin, 180, 180
blue Castro Valley, 600, 780
blue Bay Fair, 240, 1020
blue San Leandro, 240, 1260
blue Coliseum, 240, 1500
blue Fruitvale, 240, 1740
blue Lake Merritt, 300, 2040
blue West Oakland, 360, 2400
blue Embarcadero, 420, 2820
arrive Embarcadero, 0, 2820
